In [11]:
from pybaseball import statcast
import pandas as pd
import numpy as np

In [58]:
savant_df = statcast(start_dt= "2023-03-15", end_dt = "2023-08-30")

pd.set_option('display.max_columns', None)
savant_df

This is a large query, it may take a moment to complete


/Users/cbkaplinger/Library/Python/3.9/lib/python/site-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
100%|██████████| 169/169 [01:24<00:00,  2.00it/s]


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
1935,SI,2023-08-29,94.9,4.09,5.35,"Diekman, Jake",660821,518617,field_out,hit_into_play,<NA>,<NA>,<NA>,<NA>,9,"Jesus Sanchez grounds out, pitcher Jake Diekma...",R,L,L,MIA,TB,X,1,ground_ball,0,1,2023,1.51,0.38,0.68,1.86,<NA>,542932,641658,2,9,Bot,131.51,183.18,<NA>,<NA>,542194,<NA>,<NA>,-12.310629,-137.577689,-4.272845,21.94147,32.054764,-26.540252,3.5,1.67,1,62.4,-54,94.9,2098,6.6,716807,518617,542194,650490,660644,670623,678545,668227,642350,670042,53.91,0.271,0.254,0.0,1,0,0,2,83,2,Sinker,2,11,2,11,11,2,2,11,Standard,Standard,109,0.0,-0.406
1939,FF,2023-08-29,94.6,4.03,5.42,"Diekman, Jake",660821,518617,NaN,called_strike,<NA>,<NA>,<NA>,<NA>,14,"Jesus Sanchez grounds out, pitcher Jake Diekma...",R,L,L,MIA,TB,S,<NA>,NaN,0,0,2023,1.35,0.76,0.06,1.53,<NA>,542932,641658,2,9,Bot,<NA>,<NA>,<NA>,<NA>,542194,<NA>,<NA>,-13.366229,-136.997478,-6.149459,19.934123,30.230457,-21.299116,3.48,1.66,<NA>,<NA>,<NA>,94.6,2170,6.6,716807,518617,542194,650490,660644,670623,678545,668227,642350,670042,53.94,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,83,1,4-Seam Fastball,2,11,2,11,11,2,2,11,Standard,Standard,111,0.0,-0.063
1987,SI,2023-08-29,95.0,4.07,5.42,"Diekman, Jake",641658,518617,walk,ball,<NA>,<NA>,<NA>,<NA>,12,Garrett Hampson walks. Jon Berti to 2nd.,R,R,L,MIA,TB,B,<NA>,NaN,3,2,2023,1.57,0.55,0.22,3.78,<NA>,<NA>,542932,2,9,Bot,<NA>,<NA>,<NA>,<NA>,542194,<NA>,<NA>,-13.554231,-137.668289,0.193541,22.934396,30.646476,-25.519254,3.32,1.58,<NA>,<NA>,<NA>,94.8,2106,6.4,716807,518617,542194,650490,660644,670623,678545,668227,642350,670042,54.11,<NA>,<NA>,0.7,1,0,0,<NA>,82,11,Sinker,2,11,2,11,11,2,2,11,Standard,Standard,112,0.0,0.217
2053,FF,2023-08-29,95.4,4.0,5.41,"Diekman, Jake",641658,518617,NaN,foul,<NA>,<NA>,<NA>,<NA>,8,Garrett Hampson walks. Jon Berti to 2nd.,R,R,L,MIA,TB,S,<NA>,NaN,3,2,2023,1.5,0.96,0.21,1.83,<NA>,<NA>,542932,2,9,Bot,<NA>,<NA>,<NA>,<NA>,542194,<NA>,<NA>,-13.316148,-138.142463,-5.900313,22.221439,31.444874,-18.622991,3.39,1.57,181,74.0,61,95.2,2325,6.5,716807,518617,542194,650490,660644,670623,678545,668227,642350,670042,54.02,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,82,10,4-Seam Fastball,2,11,2,11,11,2,2,11,Standard,Standard,113,0.0,0.0
2063,FF,2023-08-29,95.7,3.95,5.46,"Diekman, Jake",641658,518617,NaN,foul,<NA>,<NA>,<NA>,<NA>,5,Garrett Hampson walks. Jon Berti to 2nd.,R,R,L,MIA,TB,S,<NA>,NaN,3,2,2023,1.54,0.62,0.08,2.62,<NA>,<NA>,542932,2,9,Bot,<NA>,<NA>,<NA>,<NA>,542194,<NA>,<NA>,-13.646495,-138.64782,-3.213637,22.904886,31.892111,-23.666038,3.39,1.57,204,76.6,20,95.4,2270,6.4,716807,518617,542194,650490,660644,670623,678545,668227,642350,670042,54.06,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,82,9,4-Seam Fastball,2,11,2,11,11,2,2,11,Standard,Standard,118,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [61]:
# Converts the Induced Vertical Break and Horizontal Break to inches from feet
savant_df['IVB'] = savant_df['pfx_z'] * 12
savant_df['HB'] = savant_df['pfx_x'] * 12

savant_df = savant_df.rename(columns={'player_name': 'Name', 'pitch_type': 'Pitch Type'})
savant_df

KeyError: ['pitch_type']

In [93]:
pitches_grouped = savant_df.groupby(['Name', 'Pitch Type']).agg(
    Pitches = ('Pitch Type', 'count'), 
    Average_Velo = ('release_speed', 'mean'),
    Max_Velo = ('release_speed', 'max'),
    Spin_Rate = ('release_spin_rate', 'mean'),
    Induced_Vertical_Break = ('IVB', 'mean'),
    Horizontal_Break = ('HB', 'mean'),
    Release_Height = ('release_pos_z', 'mean'),
    Extension = ('release_extension', 'mean'),
    AVG_Exit_Velo = ('launch_speed', 'mean'),
    xwOBA = ('estimated_woba_using_speedangle', 'mean'),
    Run_Value = ('delta_run_exp', 'sum'),
    SwStr_Percent = ('description', lambda x: round((x.isin(['swinging_strike', 'swinging_strike_blocked'])).mean() * 100, 2)),
    Called_Strike_Percent = ('description', lambda x: round((x == 'called_strike').mean() * 100, 2)))

pitches_grouped['CSW_Percent'] = pitches_grouped['SwStr_Percent'] + pitches_grouped['Called_Strike_Percent']
pitches_grouped

Pitches  Average_Velo  Max_Velo    Spin_Rate  \
Name              Pitch Type                                                 
Abad, Fernando    CH               28        76.625      83.6  1672.392857   
                  FF               27     93.092593      95.0  2201.925926   
                  KC               29     77.024138      79.8  2429.448276   
                  SI               94     92.458511      94.9  2135.053763   
Abbott, Andrew    CH              248     86.552419      88.5  2069.326613   
...                               ...           ...       ...          ...   
Zuñiga, Guillermo SL               17     88.052941      90.4  2643.647059   
deGrom, Jacob     CH               40        91.405      92.8     1670.525   
                  CU                9     84.755556      87.1       2749.0   
                  FF              235     98.711915     101.2  2550.029787   
                  SL              167     91.776647      95.4  2643.694611   

                              Induced_Vertical_Break  Horizontal_Break  \
Name              Pitch Type                                             
Abad, Fernando    CH                           12.75         10.191429   
                  FF                       14.355556          7.253333   
                  KC                       -6.881379         -7.402759   
                  SI                        11.51617         12.917872   
Abbott, Andrew    CH                       10.922903         14.596935   
...                                              ...               ...   
Zuñiga, Guillermo SL                       -0.621176          4.814118   
deGrom, Jacob     CH                           3.942           -14.307   
                  CU                       -3.413333              6.64   
                  FF                       17.553702         -8.694638   
                  SL                        3.847186          4.778443   

                              Release_Height  Extension  AVG_Exit_Velo  \
Name              Pitch Type                                             
Abad, Fernando    CH                6.399643   6.228571         82.525   
                  FF                6.216667   5.822222          71.27   
                  KC                6.301724   5.651724      80.358333   
                  SI                 6.26617   5.842553      83.835294   
Abbott, Andrew    CH                5.778105    6.72621       78.65303   
...                                      ...        ...            ...   
Zuñiga, Guillermo SL                5.998824   6.194118      81.671429   
deGrom, Jacob     CH                 5.59925     6.8625      78.271429   
                  CU                5.751111   6.544444          88.45   
                  FF                5.610979    6.80383      84.818056   
                  SL                5.679102   6.624551      78.573684   

                                 xwOBA  Run_Value  SwStr_Percent  \
Name              Pitch Type                                       
Abad, Fernando    CH          0.552833      2.113           7.14   
                  FF            0.2346       0.23           7.41   
                  KC           0.43325      -0.55           3.45   
                  SI          0.470556      0.681           4.26   
Abbott, Andrew    CH          0.305842     -3.132          16.53   
...                                ...        ...            ...   
Zuñiga, Guillermo SL             0.817     -0.182          11.76   
deGrom, Jacob     CH          0.305571     -1.862          17.50   
                  CU             0.266     -0.383          22.22   
                  FF          0.319684     -1.914          16.17   
                  SL          0.340842     -0.837          26.95   

                              Called_Strike_Percent  CSW_Percent  
Name              Pitch Type                                      
Abad, Fernando    CH                          14.29        21.43  
   

In [98]:
player_grouped = savant_df.groupby(['Name']).agg(
    Pitches = ('Pitch Type', 'count'),
    BB = ('events', lambda x: (x == 'walk').sum()),
    HBP = ('events', lambda x: (x == 'hit_by_pitch').sum()),
    K = ('events', lambda x: (x == 'strikeout').sum()),
    GB = ('bb_type', lambda x: (x == 'ground_ball').sum()),
    BIP = ('bb_type', lambda x: (x.isin(['ground_ball', 'line_drive', 'fly_ball', 'popup'])).sum()),
    xwOBA = ('estimated_woba_using_speedangle', 'mean'),
    Run_Value = ('delta_run_exp', 'sum'),
    SwStr_Percent = ('description', lambda x: round((x.isin(['swinging_strike', 'swinging_strike_blocked'])).mean() * 100, 2)),
    Called_Strike_Percent = ('description', lambda x: round((x == 'called_strike').mean() * 100, 2)))

player_grouped['CSW%'] = player_grouped['SwStr_Percent'] + player_grouped['Called_Strike_Percent']
player_grouped['GB%'] = round((player_grouped['GB'] / player_grouped['BIP']) * 100, 2)

player_grouped

,Pitches,BB,HBP,K,GB,BIP,xwOBA,Run_Value,SwStr_Percent,Called_Strike_Percent,CSW%,GB%
Name,,,,,,,,,,,,
"Abad, Fernando",178,3,0,5,11,33,0.445242,2.474,5.06,17.98,23.04,33.33
"Abbott, Andrew",1505,34,0,98,65,232,0.376246,-9.72,10.83,15.95,26.78,28.02
"Abbott, Cory",526,17,3,25,41,88,0.373864,9.23,10.65,16.92,27.57,46.59
"Abney, Alaska",4,0,0,0,0,1,0.006,0.0,0.00,25.00,25.00,0.00
"Abreu, Albert",1081,32,6,57,80,176,0.370626,4.121,9.44,17.95,27.39,45.45
...,...,...,...,...,...,...,...,...,...,...,...,...
"Zastryzny, Rob",374,11,0,18,27,65,0.342277,3.159,11.76,18.45,30.21,41.54
"Zerpa, Angel",313,4,4,14,28,59,0.358136,7.011,9.58,11.82,21.40,47.46
"Zimmermann, Bruce",187,0,2,12,19,39,0.392846,1.482,16.04,16.58,32.62,48.72


In [90]:
savant_df['events'].unique()

array(['field_out', nan, 'walk', 'single', 'strikeout', 'home_run',
       'grounded_into_double_play', 'double', 'hit_by_pitch',
       'caught_stealing_2b', 'sac_fly', 'force_out', 'field_error',
       'triple', 'sac_bunt', 'caught_stealing_3b', 'fielders_choice',
       'double_play', 'fielders_choice_out', 'sac_fly_double_play',
       'strikeout_double_play', 'catcher_interf', 'stolen_base_3b',
       'pickoff_2b', 'pickoff_1b', 'triple_play', 'pickoff_error_3b',
       'pickoff_caught_stealing_2b', 'other_out', 'caught_stealing_home',
       'wild_pitch', 'pickoff_3b', 'pickoff_caught_stealing_home',
       'stolen_base_2b'], dtype=object)